### Construct a binary [16,8,5] Goppa code (Example 15.31)

Section 15.3 of "A Course in Cryptography"

Author: H. Knospe

In [1]:
K.<z>=GF(2^4, name='z', modulus=x^4+x+1) # K=GF(16)

In [2]:
PR.<x>=K[] # polynomial ring K[x]

In [3]:
g=x^2+z^2*x+z; g.is_irreducible() # irreducible polynomial g of degree 2 over GF(16)

True

In [4]:
Rmodg=PR.quotient_ring(g) # GF(16)/(g(x))

In [5]:
arr=[]  # define values 1/(x-a) for a in K
for a in K.list():
	arr.append(1/Rmodg(x-a)) 

In [6]:
H16=matrix(K,2,16)

In [7]:
for i in range(0,2):
	for j in range(0,16):
		H16[i,j]=list(arr[j])[i]

In [8]:
H16 # parity check matrix of a [16,14] code C over GF(16)

[                z           z^3 + z                 0               z^3               z^3           z^2 + z                 1 z^3 + z^2 + z + 1           z^2 + z           z^3 + z z^3 + z^2 + z + 1       z^3 + z + 1       z^3 + z + 1                 z                 1     z^3 + z^2 + 1]
[          z^3 + 1             z + 1           z^3 + 1 z^3 + z^2 + z + 1           z^2 + 1             z + 1 z^3 + z^2 + z + 1                 1           z^2 + z     z^3 + z^2 + 1           z^2 + 1     z^3 + z^2 + 1                 1               z^2               z^2           z^2 + z]

In [9]:
H=matrix(GF(2),8,16)
for i in range(0,2):
	for j in range(0,16):
		H16[i,j]=list(arr[j])[i]
		hbin=bin(eval(H16[i,j]._int_repr()))[2:]
		hbin = '0'*(4-len(hbin))+hbin; hbin = list(hbin);
		H[4*i:4*(i+1),j] = vector(map(GF(2),hbin));

In [10]:
H # parity check matrix of associated [16,8,5] Goppa code over GF(2)

[0 1 0 1 1 0 0 1 0 1 1 1 1 0 0 1]
[0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 1]
[1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0]
[0 0 0 0 0 0 1 1 0 0 1 1 1 0 1 1]
[1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0]
[0 0 0 1 1 0 1 0 1 1 1 1 0 1 1 1]
[0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 1]
[1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0]

In [11]:
G=H.right_kernel().basis_matrix()

In [12]:
G # generator matrix of the Goppa code

[1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1]
[0 1 0 0 0 0 0 0 1 0 1 0 0 1 1 0]
[0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1]
[0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 0]
[0 0 0 0 1 0 0 0 0 1 0 1 1 1 0 0]
[0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1]
[0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 1]
[0 0 0 0 0 0 0 1 0 1 1 1 0 0 1 0]

In [13]:
Gamma = LinearCode(G)

In [14]:
Gamma.minimum_distance() # SageMath verifies that the minimum distance is 5

5

### Decoding using Patterson's algorithm (Example 15.32)

Section 15.3 of "A Course in Cryptography"

Author: H. Knospe

In [15]:
v=vector(GF(2),[1, 1, 0, 1, 0, 0, 1, 0])*G # codeword

In [16]:
e=vector(GF(2),[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]) # error positions are 3 and 7

In [17]:
w=v+e # erroneous codeword

In [18]:
syn=w*vector(arr); syn # syndrome Syn(w)

(z^2 + z)*xbar + 1

In [19]:
T=1/(w*vector(arr)); T # 1/Syn(w)

z^3*xbar + z^3 + z + 1

In [20]:
T-Rmodg(x)

(z^3 + 1)*xbar + z^3 + z + 1

In [21]:
R=(T-Rmodg(x))^128; R # sqrt(T-x) in GF(16)/(g)

(z^3 + z^2)*xbar + z^2 + z + 1

In [22]:
a0=(z^3+z^2)*x+z^2+z+1;b0=1
sigma=a0*a0+x*b0*b0; sigma # lift R to K[x] and set sigma=a0^2 + x*b0^2

(z^3 + z^2 + z + 1)*x^2 + x + z^2 + z

In [23]:
sigma.factor()

(z^3 + z^2 + z + 1) * (x + z^2) * (x + z^3 + z^2)

In [24]:
i=0; e=vector(GF(2),16)

In [25]:
for k in list(K):
    if ((sigma.subs(x=k))==0):
        e[i]=1
    i=i+1
print(e)        # print error vector

(0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [26]:
e*vector(arr) # Syn(e); w and e have the same syndrome

(z^2 + z)*xbar + 1

In [27]:
w*H.transpose()

(0, 0, 0, 1, 0, 1, 1, 0)

In [28]:
e*H.transpose()

(0, 0, 0, 1, 0, 1, 1, 0)

### McEliece (Example 15.34)

Section 15.4 of "A Course in Cryptography"

Author: H. Knospe

In [29]:
S=matrix(GF(2),
[[0, 0, 1, 0, 0, 1, 1, 0],
 [1, 0, 1, 0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 0, 1, 0, 1],
 [1, 1, 1, 1, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 1, 0, 1],
 [0, 0, 1, 1, 1, 1, 1, 0]]) # secret invertible 8x8 matrix

In [30]:
perm = Permutation([11,13,3,8,15,5,2,1,9,7,6,16,4,10,12,14])
P = matrix(GF(2),perm.to_matrix());P # secret 16x16 permutation matrix

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]

In [31]:
G1 = S*G*P; G1 # public generator matrix of a [16,8,5] code

[1 0 1 0 1 0 0 0 1 1 1 1 0 0 0 1]
[1 0 1 0 0 1 0 1 0 1 0 1 0 0 1 1]
[0 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0]
[1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0]
[1 0 0 1 1 0 1 0 0 0 1 1 1 1 0 1]
[0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 1]
[0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0]
[0 0 1 0 1 1 0 0 0 1 1 1 1 0 1 0]

In [32]:
v = vector(GF(2),[0, 1, 1, 1, 0, 0, 1, 1]) # plaintext

In [33]:
e = vector(GF(2),[0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0]) # error vector

In [34]:
y=v*G1 + e; y # ciphertext

(0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1)

In [35]:
y1=y*P.inverse(); y1 # first decryption step

(0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1)

In [36]:
# Now Goppa decoding using Patterson's algorithm 

def patterson(w):
    e=vector(GF(2),16)
    Syn=w*vector(arr); Syn # syndrome Syn(w)
    T=1/(w*vector(arr))
    if (Syn==0):
        return(e,w) # no error
    if (T==Rmodg(x)):
        sigma=x
    else:
        R=(T-Rmodg(x))^128 # sqrt(T-x) in GF(16)/(g)
        a0=R.lift() # lift R to K[x]
        b0=1
        sigma=a0*a0+x*b0*b0 # set sigma=a0^2 + x*b0^2    
    i=0    
    for k in list(K):
        if ((sigma.subs(x=k))==0):
            e[i]=1
        i=i+1
    return(e,w+e) # error vector and codeword

In [37]:
e, c = patterson(y1); print(w); print(e); print(c) # Exercise 10

(1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0)
(0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1)


In [38]:
G.solve_left(c)*S.inverse() # recover plaintext

(0, 1, 1, 1, 0, 0, 1, 1)

### Information-set Decoding (Example 15.35)

Section 15.4 of "A Course in Cryptography"

Author: H. Knospe

In [39]:
GI=G1[:,3:11];GI # index set I: 4,5,6,7,8,9,10,11

[0 1 0 0 0 1 1 1]
[0 0 1 0 1 0 1 0]
[0 1 0 0 0 0 0 1]
[0 1 0 1 1 0 0 0]
[1 1 0 1 0 0 0 1]
[0 0 1 1 1 0 0 0]
[1 0 0 0 0 1 0 1]
[0 1 1 0 0 0 1 1]

In [40]:
yI=y[3:11]; yI # extract positions I from the ciphertext

(1, 1, 0, 1, 0, 1, 0, 1)

In [41]:
xI=yI*GI.inverse(); xI # success; get plaintext since error is outside I

(0, 1, 1, 1, 0, 0, 1, 1)

In [42]:
xI*G1+y # verfication ok, only two errors

(0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0)

In [43]:
GI=G1[:,0:8];GI # other index set I: 1,2,3,4,5,6,7,8

[1 0 1 0 1 0 0 0]
[1 0 1 0 0 1 0 1]
[0 1 0 0 1 0 0 0]
[1 0 0 0 1 0 1 1]
[1 0 0 1 1 0 1 0]
[0 0 1 0 0 1 1 1]
[0 0 0 1 0 0 0 0]
[0 0 1 0 1 1 0 0]

In [44]:
yI=y[0:8]; yI # extract positions I from the ciphertext

(0, 0, 0, 1, 1, 0, 1, 0)

In [45]:
xI=yI*GI.inverse(); xI # now positions I contain errors

(0, 0, 0, 1, 1, 1, 0, 1)

In [46]:
xI*G1+y # verfication fails, 6 errors

(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1)

### Niederreiter Cryptosystem (Example 15.37)

Section 15.4 of "A Course in Cryptography"

Author: H. Knospe

In [47]:
w = vector(GF(2),[0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0]) # plaintext vector of weight 2

In [48]:
y = S*H*P*w; y # ciphertext

(0, 0, 1, 0, 1, 1, 1, 0)

In [49]:
syn=S.inverse()*y; syn # syndrome (Exercise 12)

(0, 0, 1, 1, 1, 1, 0, 1)

In [50]:
z=H.solve_right(syn);z  # a solution of Hz^T = syn

(1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0)

In [51]:
# Now Goppa decoding as above 
e, c = patterson(z); print(z) ; print(e) ; print(c)

(1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0)
(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0)


In [52]:
P.inverse()*e # recover the plaintext 

(0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0)